In [ ]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.optimize import Bounds
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from collections import namedtuple
from matplotlib import cm
from scipy.stats import mannwhitneyu
import cma
from scipy.optimize import minimize
from scipy.optimize import Bounds
import sys
from sklearn.preprocessing import MinMaxScaler
import pyDOE
import pandas as pd
import numpy as np
import scipy.stats.distributions as dist
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from sklearn.ensemble import RandomForestRegressor
from scipy.optimize import minimize
from scipy.optimize import Bounds
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from collections import namedtuple
from sklearn.linear_model import ElasticNet
from RBFN import RBFN

## Helper Functions

In [2]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' Plot the Graph of the Function'''
def plot_the_graph(x,y,z,graph_title):
    fig = plt.figure(figsize=(12,5))
    ax = Axes3D(fig)
    surf = ax.plot_trisurf(x,y,z,cmap=cm.jet, linewidth=0.3,alpha=0.9)
    fig.colorbar(surf, shrink=10, aspect=3)
    ax.set_xlabel('X1')
    ax.set_ylabel('X2')
    ax.set_zlabel('Robust(X1,X2)')
    plt.title(graph_title)
    plt.show()
    fig.savefig(graph_title+'.pdf')

''' Branin Test Example  '''
def branin_test_example(X):
    x,z = X
    A = (z) - ((5.1) / (4 * np.square(np.pi))) * (x **2) + (( (5)/(np.pi) ) * (x) ) - (6)
    B = (10 * ((1) - ((1) / (8 * np.pi) )) ) * (np.cos(x))
    return np.square(A) + B + 10


''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=2, samples=n_obs, criterion='m')
    X1, X2 = lhd[:,0], lhd[:,1]
    return X1,X2


'''Robust Regularization based on minimax Principle 5% perturbations'''
def robust_regularization(X):
    x1,x2 = X
    eps = np.linspace(-0.75,0.75,1000)
    return np.max( branin_test_example([x1+eps,x2+eps]) )

''' Robustness based on a composite function of Mean and STd '''
def composite_robustness(X):
    w=1
    x1,x2 = X
    np.random.seed(0)
    eps = np.random.normal(loc=0.0,scale=0.125,size=10000)
    sample_points = branin_test_example([x1+eps,x2+eps])
    sample_mean = np.mean(sample_points)
    variance = np.square(sample_points-sample_mean)
    std = np.sqrt(np.mean(variance))
    std = np.sqrt(np.mean(variance))
    return sample_mean + w * std

''' Generate Training Data using LHD along side the Output of the Robust System'''
def generate_training_data(n_obs):
    X1,X2 = DOE(n_obs)
    X1 = linearscaletransform(X1,range_out=(-5,10))
    X2 = linearscaletransform(X2,range_out=(0,15))
    f_evaluation = branin_test_example([X1,X2])
    f_original = np.zeros(X1.shape[0])
    minimax_original = np.zeros(X1.shape[0])
    composite_original = np.zeros(X1.shape[0])
    for i in range(X1.shape[0]):
        f_original[i] = branin_test_example([X1[i],X2[i]])
        minimax_original[i] = robust_regularization([X1[i],X2[i]])
        composite_original[i] = composite_robustness([X1[i],X2[i]])
    train = pd.DataFrame()
    train['X1'] = pd.Series(X1)
    train['X2'] = pd.Series(X2)
    train['Y = F(X1,X2)'] = pd.Series(f_original)
    train['Y = robust_regularization(X1,X2)'] = pd.Series(minimax_original)
    train['Y = composite(X1,X2)'] = pd.Series(composite_original)
    train.to_csv('Training_Data_Sets\\'+str(n_obs)+'Samples.csv')
    return train

''' Generate Test Data using LHD along side the Output of the Robust System'''
def generate_test_data(n_obs):
    test = pd.DataFrame()
    np.random.seed(0)
    lhd = pyDOE.lhs(n=2, samples=n_obs, criterion='m')
    X1 = linearscaletransform(lhd[:,0],range_out=(-5,10))
    X2 = linearscaletransform(lhd[:,1],range_out=(0,15))
    test ['X1'] = pd.Series(X1)
    test ['X2'] = pd.Series(X2)
    true_minimax = np.zeros(test.shape[0])
    true_composite = np.zeros(test.shape[0])
    for i in range(test.shape[0]):
        true_minimax[i] = robust_regularization(test.iloc[i,:])
        true_composite[i] = composite_robustness(test.iloc[i,:])
    test ['True_Minimax'] = pd.Series(true_minimax)
    test ['True_Composite'] = pd.Series(true_composite)
    test.to_csv('Test_Data_Sets\\'+str(n_obs)+'Samples.csv')
    return test

''' Check if There are same examples in Test and Train Data Sets'''
def check_data_sets(train,test):
    C = train.isin(test)
    C = pd.DataFrame.sum(C,axis=1)
    ids = C.index[C>1]
    print ('Total Number of Input Observations present in Test Data : '+str(ids.shape[0]))

## Meta-Models

In [3]:
''' Elastic Net Regression for Minimax Robustness'''
def elastic_net_minimax(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:5].values, test_data.values])
    regr=ElasticNet(alpha= 0.03471316333696191   ,random_state=0 , l1_ratio= 0.0, fit_intercept =True, max_iter=1000,selection='random'
                   ).fit(scaler.transform(train_data.iloc[:,:5]), train_data.iloc[:,6])
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = np.array([x[0] , x[1] , x[0] * x[1] , x[0] **2 , x[1] **2])
            x = np.atleast_2d(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)

''' Elastic Net Regression for Minimax Robustness'''
def elastic_net_composite(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:5].values, test_data.values])
    regr=ElasticNet(alpha= 0.03562626988750126  ,random_state=0 , l1_ratio= 0.0, fit_intercept =True, max_iter=1000,selection='random'
                   ).fit(scaler.transform(train_data.iloc[:,:5]), train_data.iloc[:,-1])
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = np.array([x[0] , x[1] , x[0] * x[1] , x[0] **2 , x[1] **2])
            x = np.atleast_2d(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)


''' Kriging for Robust Regularization'''
def kriging_minimax(train_data,test_data):
    kernel =  RBF(78, (2, 75065))
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:2].values, test_data.values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer=89,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:2]), train_data.iloc[:,3])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Kriging for Robust Composition'''
def kriging_composite(train_data,test_data):
    kernel =  RBF(45, (1, 14296))
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:2].values, test_data.values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer=93,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:2]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Random Forest Implementation for Minimax Robustness'''
def rf_minimax(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:2].values, test_data.values])
    regr = RandomForestRegressor(random_state=42,n_estimators=6
                                ).fit(scaler.transform(train_data.iloc[:,:2]), train_data.iloc[:,3])
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler,regr)

''' Random Forest Implementation for Composite Robustness'''
def rf_composite(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:2].values, test_data.values])
    regr = RandomForestRegressor(random_state=31,n_estimators=5
                                ).fit(scaler.transform(train_data.iloc[:,:2]), train_data.iloc[:,-1])
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler,regr)


''' KNN Regression Implementation for Minimax Robustness'''
def KNN_minimax(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:2].values, test_data.values])
    regr = KNeighborsRegressor(n_neighbors=3,weights='distance',algorithm='brute',p=2
                               ).fit(scaler.transform(train_data.iloc[:,:2]), train_data.iloc[:,3])

    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler,regr)

''' KNN Regression Implementation for Composite Robustness'''
def KNN_composite(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:2].values, test_data.values])
    regr = KNeighborsRegressor(n_neighbors=2,weights='distance',algorithm='brute',p=2
                               ).fit(scaler.transform(train_data.iloc[:,:2]), train_data.iloc[:,-1])

    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler,regr)

''' Support Vector Regression for Robust Regularization'''
def SVR_minimax(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:2].values, test_data.values])
    gpr = SVR(gamma =  2.311373    ,C =  651.399134, epsilon=0.2,max_iter=1500).fit(
        scaler.transform(train_data.iloc[:,:2]), train_data.iloc[:,3])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

''' Support Vector Regression for Robust Composition'''
def SVR_composite(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:2].values, test_data.values])
    gpr = SVR(gamma =  0.963224,C = 606.227521, epsilon=0.2,max_iter=1500).fit(
        scaler.transform(train_data.iloc[:,:2]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' RBF Network Interpolation for Robust Regularization'''
def RBF_minimax(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:2].values, test_data.values])
    model = RBFN(hidden_shape=68, sigma= 1.469632)
    model.fit(scaler.transform(np.array(train_data.iloc[:,:2])), np.array(train_data.iloc[:,3]))
    pred = model.predict(scaler.transform(np.array(test_data)))
    def predict(scaler, model):
        def __predict__(x):
            x = np.atleast_2d(x)
            return model.predict(scaler.transform(x))
        return __predict__
    return model,pred, predict(scaler,model)

''' RBF Network Interpolation for Robust Composition'''
def RBF_composite(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:2].values, test_data.values])
    model = RBFN(hidden_shape=68, sigma= 1.469632)
    model.fit(scaler.transform(np.array(train_data.iloc[:,:2])), np.array(train_data.iloc[:,-1]))
    pred = model.predict(scaler.transform(np.array(test_data)))
    def predict(scaler, model):
        def __predict__(x):
            x = np.atleast_2d(x)
            return model.predict(scaler.transform(x))
        return __predict__
    return model,pred, predict(scaler,model)

In [ ]:
train = pd.read_csv(sys.path[1]+str('\\Training_Data_Sets\\100Samples.csv')).iloc[:,1:]
test = pd.read_csv('Optimization_Test.csv').iloc[:,1:]
model_k_m,pred_k_m, predict_k_m = kriging_minimax(train,test.iloc[:,:2])
model_k_c,pred_k_c, predict_k_c = kriging_composite(train,test.iloc[:,:2])
model_r_m,pred_r_m, predict_r_m = rf_minimax(train,test.iloc[:,:2])
model_r_c,pred_r_c, predict_r_c = rf_composite(train,test.iloc[:,:2])
model_n_m,pred_n_m, predict_n_m = KNN_minimax(train,test.iloc[:,:2])
model_n_c,pred_n_c , predict_n_c= KNN_composite(train,test.iloc[:,:2])
model_s_m,pred_s_m, predict_s_m = SVR_minimax(train,test.iloc[:,:2])
model_s_c,pred_s_c, predict_s_c = SVR_composite(train,test.iloc[:,:2])
model_b_m,pred_b_m, predict_b_m = RBF_minimax(train,test.iloc[:,:2])
model_b_c,pred_b_c, predict_b_c = RBF_composite(train,test.iloc[:,:2])
train ['X1X2'] = pd.Series( np.array(train.X1) * np.array(train.X2) )
train ['X1**2'] = pd.Series ( np.array(train.X1)**2 )
train ['X2**2'] = pd.Series ( np.array(train.X2)**2 )
f_original = train['Y = F(X1,X2)']
robust_original = train['Y = robust_regularization(X1,X2)']
composite_original = train ['Y = composite(X1,X2)']
del train['Y = F(X1,X2)']
del train['Y = robust_regularization(X1,X2)']
del train ['Y = composite(X1,X2)']
train['Y = F(X1,X2)'] = f_original
train['Y = robust_regularization(X1,X2)'] = robust_original
train ['Y = composite(X1,X2)'] = composite_original
test ['X1X2'] =  pd.Series( np.array(test.X1) * np.array(test.X2) )
test ['X1**2'] = pd.Series ( np.array(test.X1)**2 )
test ['X2**2'] = pd.Series ( np.array(test.X2)**2 )
model_m_m,pred_m_m, predict_m_m = elastic_net_minimax(train,test.iloc[:,:5])
model_m_c,pred_m_c, predict_m_c = elastic_net_composite(train,test.iloc[:,:5])
del train['X1X2']
del train['X1**2']
del train['X2**2']
del test['X1X2']
del test['X1**2']
del test['X2**2']

In [5]:
Columns = ['Benchmark', 'Kriging' , 'SVM' , 'RBFN' ,  'KNN' , 'RF'  , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,3):
        Cols.append(Columns[j]+'_X'+str(i))
        
const = Bounds([-5, 0], [10, 15])
G1,G2 = DOE(100)
G1 = linearscaletransform(G1,range_out=(-5,10))
G2 = linearscaletransform(G2,range_out=(0,15))


X_Values_RR = np.zeros([100,14])
X_Values_RC = np.zeros([100,14])
Fun_Eval_RR = np.zeros([100,7])
Fun_Eval_RC = np.zeros([100,7])

## SLSQP

In [6]:
for i in range(X_Values_RR.shape[0]):
    min_robust = minimize(robust_regularization,(np.array([G1[i],G2[i]])),method='SLSQP',bounds=const)
    min_compo = minimize(composite_robustness,(np.array([G1[i],G2[i]])),method='SLSQP',bounds=const)
    min_robust_k = minimize(predict_k_m,(np.array([G1[i],G2[i]])),method='SLSQP',bounds=const)
    min_composite_k = minimize(predict_k_c,(np.array([G1[i],G2[i]])),method='SLSQP',bounds=const)
    min_robust_s = minimize(predict_s_m,(np.array([G1[i],G2[i]])),method='SLSQP',bounds=const)
    min_composite_s = minimize(predict_s_c,(np.array([G1[i],G2[i]])),method='SLSQP',bounds=const)
    min_robust_b = minimize(predict_b_m,(np.array([G1[i],G2[i]])),method='SLSQP',bounds=const)
    min_composite_b = minimize(predict_b_c,(np.array([G1[i],G2[i]])),method='SLSQP',bounds=const)
    min_robust_n = minimize(predict_n_m,(np.array([G1[i],G2[i]])),method='SLSQP',bounds=const)
    min_composite_n = minimize(predict_n_c,(np.array([G1[i],G2[i]])),method='SLSQP',bounds=const)
    min_robust_r = minimize(predict_r_m,(np.array([G1[i],G2[i]])),method='SLSQP',bounds=const)
    min_composite_r = minimize(predict_r_c,(np.array([G1[i],G2[i]])),method='SLSQP',bounds=const)
    min_robust_m = minimize(predict_m_m,(np.array([G1[i],G2[i]])),method='SLSQP',bounds=const)
    min_composite_m = minimize(predict_m_c,(np.array([G1[i],G2[i]])),method='SLSQP',bounds=const)
    
   
    
    
    
    X_Values_RR [i,:] = list(min_robust.x)+list(min_robust_k.x)+list(min_robust_s.x)+list(min_robust_b.x)+list(min_robust_n.x)+list(min_robust_r.x)+list(min_robust_m.x)
    
    X_Values_RC [i,:] = list(min_compo.x)+list(min_composite_k.x)+list(min_composite_s.x)+list(min_composite_b.x)+list(min_composite_n.x)+list(min_composite_r.x)+list(min_composite_m.x)
    
    Fun_Eval_RR [i,:] = [min_robust.nfev]+[100,100,100,100,100,100]
    
    Fun_Eval_RC [i,:] = [min_compo.nfev]+[100,100,100,100,100,100]

In [7]:
X_Values_RR = pd.DataFrame(X_Values_RR)
X_Values_RR.columns = Cols
X_Values_RC = pd.DataFrame(X_Values_RC)
X_Values_RC.columns = Cols
Fun_Eval_RR = pd.DataFrame(Fun_Eval_RR)
Fun_Eval_RR.columns = Columns
Fun_Eval_RC = pd.DataFrame(Fun_Eval_RC)
Fun_Eval_RC.columns = Columns

In [8]:
X_Values_RR = pd.DataFrame(X_Values_RR)
X_Values_RR.columns = Cols
X_Values_RC = pd.DataFrame(X_Values_RC)
X_Values_RC.columns = Cols
Fun_Eval_RR = pd.DataFrame(Fun_Eval_RR)
Fun_Eval_RR.columns = Columns
Fun_Eval_RC = pd.DataFrame(Fun_Eval_RC)
Fun_Eval_RC.columns = Columns

## Meta-Modelling

In [9]:
X_Values_RR.to_csv('Results\\X_Values_RR.csv')
X_Values_RC.to_csv('Results\\X_Values_RC.csv')
Fun_Eval_RR.to_csv('Results\\Function_Eval_RR.csv')
Fun_Eval_RC.to_csv('Results\\Function_Eval_RC.csv')
RR = pd.read_csv('Results\\X_Values_RR.csv').iloc[:,1:]
RC = pd.read_csv('Results\\X_Values_RC.csv').iloc[:,1:]

In [10]:
Bench_Fun_RR = np.zeros(100)
Krig_Fun_RR = np.zeros(100)
SVM_Fun_RR = np.zeros(100)
RBFN_Fun_RR = np.zeros(100)
KNN_Fun_RR = np.zeros(100)
RF_Fun_RR = np.zeros(100)
ELN_Fun_RR = np.zeros(100)
for i in range(X_Values_RR.shape[0]):
    Bench_Fun_RR [i] = robust_regularization(X_Values_RR.iloc[i,:2])
    Krig_Fun_RR [i] = robust_regularization(X_Values_RR.iloc[i,2:4])
    SVM_Fun_RR [i] = robust_regularization(X_Values_RR.iloc[i,4:6])
    RBFN_Fun_RR [i] = robust_regularization(X_Values_RR.iloc[i,6:8])
    KNN_Fun_RR [i] = robust_regularization(X_Values_RR.iloc[i,8:10])
    RF_Fun_RR [i] = robust_regularization(X_Values_RR.iloc[i,10:12])
    ELN_Fun_RR [i] = robust_regularization(X_Values_RR.iloc[i,12:14])

In [11]:
print ('SLSQP')
print (stats.mode(Bench_Fun_RR) , np.std(Bench_Fun_RR))
print ('Kriging')
print (stats.mode(Krig_Fun_RR) , np.std(Krig_Fun_RR))
print ('SVM')
print (stats.mode(SVM_Fun_RR) , np.std(SVM_Fun_RR))
print ('RBFN')
print (stats.mode(RBFN_Fun_RR) , np.std(RBFN_Fun_RR))
print ('KNN')
print (stats.mode(KNN_Fun_RR) , np.std(KNN_Fun_RR))
print ('RF')
print (stats.mode(RF_Fun_RR) , np.std(RF_Fun_RR))
print ('ELN')
print (stats.mode(ELN_Fun_RR) , np.std(ELN_Fun_RR))

SLSQP
ModeResult(mode=array([2.98801356]), count=array([1])) 2.507538081362997
Kriging
ModeResult(mode=array([5.58386601]), count=array([1])) 63.885873796683875
SVM
ModeResult(mode=array([8.60577697]), count=array([1])) 2.5112630262448628
RBFN
ModeResult(mode=array([5.51854611]), count=array([1])) 63.685317784486585
KNN
ModeResult(mode=array([5.72607585]), count=array([1])) 52.066389958444155
RF
ModeResult(mode=array([5.72608917]), count=array([1])) 63.920402713192466
ELN
ModeResult(mode=array([17.08883032]), count=array([42])) 1.6067237221187526e-10


In [12]:
Bench_Fun_RC = np.zeros(100)
Krig_Fun_RC = np.zeros(100)
SVM_Fun_RC = np.zeros(100)
RBFN_Fun_RC = np.zeros(100)
KNN_Fun_RC = np.zeros(100)
RF_Fun_RC = np.zeros(100)
ELN_Fun_RC = np.zeros(100)
for i in range(X_Values_RC.shape[0]):
    Bench_Fun_RC [i] = composite_robustness(X_Values_RC.iloc[i,:2])
    Krig_Fun_RC [i] = composite_robustness(X_Values_RC.iloc[i,2:4])
    SVM_Fun_RC [i] = composite_robustness(X_Values_RC.iloc[i,4:6])
    RBFN_Fun_RC [i] = composite_robustness(X_Values_RC.iloc[i,6:8])
    KNN_Fun_RC [i] = composite_robustness(X_Values_RC.iloc[i,8:10])
    RF_Fun_RC [i] = composite_robustness(X_Values_RC.iloc[i,10:12])
    ELN_Fun_RC [i] = composite_robustness(X_Values_RC.iloc[i,12:14])

In [13]:
print ('SLSQP')
print (stats.mode(Bench_Fun_RC) , np.std(Bench_Fun_RC))
print ('Kriging')
print (stats.mode(Krig_Fun_RC) , np.std(Krig_Fun_RC))
print ('SVM')
print (stats.mode(SVM_Fun_RC) , np.std(SVM_Fun_RC))
print ('RBFN')
print (stats.mode(RBFN_Fun_RC) , np.std(RBFN_Fun_RC))
print ('KNN')
print (stats.mode(KNN_Fun_RC) , np.std(KNN_Fun_RC))
print ('RF')
print (stats.mode(RF_Fun_RC) , np.std(RF_Fun_RC))
print ('ELN')
print (stats.mode(ELN_Fun_RC) , np.std(ELN_Fun_RC))

SLSQP
ModeResult(mode=array([0.57358994]), count=array([1])) 0.1713733936429688
Kriging
ModeResult(mode=array([1.39092599]), count=array([1])) 52.02764455084981
SVM
ModeResult(mode=array([3.18506436]), count=array([1])) 6.940028448290786
RBFN
ModeResult(mode=array([1.34171041]), count=array([1])) 51.41937643033686
KNN
ModeResult(mode=array([1.50876349]), count=array([1])) 42.873348759793934
RF
ModeResult(mode=array([1.50877233]), count=array([1])) 52.317147212648294
ELN
ModeResult(mode=array([11.66358796]), count=array([45])) 1.0441285005381706e-10
